<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1_cross_validation_Francesca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff

import numpy as np
from numpy import savetxt

import pandas as pd
from pandas import DataFrame
from pandas import read_csv
from pandas import concat

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, LeakyReLU, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence

from sklearn.utils import compute_class_weight
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from matplotlib.pyplot import legend

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
def generate_model(shape):
  
  model = Sequential()
  model.add(BatchNormalization())
  
  model.add(Dense(57, input_dim=shape, use_bias=False, kernel_initializer='random_uniform', activation='relu'))
  model.add(BatchNormalization())
  #model.add(Activation('relu'))
  model.add(Dropout(0.5))  
  
  model.add(Dense(113, use_bias=False, kernel_initializer='random_uniform', activation='relu'))
  model.add(BatchNormalization())
  #model.add(Activation('relu'))
  model.add(Dropout(0.5))
 
  '''model.add(Dense(153, use_bias=False, kernel_initializer='random_uniform'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.5))'''
  
  model.add(Dense(1, activation='sigmoid'))
 
  #print(model.summary())

  return model

In [64]:
#LOAD DATA
print("Loading data...")

dataset = '/root/data/6_uHoo_featureDataset.arff'

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("Converting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

Loading data...
DATASET LOADED
Converting values...
CONVERSION DONE


In [65]:
#OPTIMIZERS
sgd = optimizers.SGD(lr=0.0001)
adm = optimizers.Adam(lr=0.0001)
rms = optimizers.RMSprop(lr=0.001)

#OTHER OPTIMIZERS
adagrad = optimizers.Adagrad(lr=0.0001)
adadelta = optimizers.Adadelta(lr=0.0001)
adamax = optimizers.Adamax(lr=0.0001)
nadam = optimizers.Nadam(lr=0.0001)

#DEFINE 10-FOLD CROSS-VALIDATION
kfold = KFold(n_splits=6, shuffle=False)
cvscores = []
predictions = []

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  model = generate_model(X[train].shape[-1])

  #COMPILE MODEL
  model.compile(loss='binary_crossentropy', optimizer = adm , metrics=['accuracy'])
  
  #EARLY STOPPING
  #es = EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=1, mode='auto')

  #FIT MODEL
  history = model.fit(X[train], Y[train], epochs=20, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)#, callbacks = [es])

  #EVALUATE MODEL
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 1)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  pred = model.predict_classes(X[test], batch_size=128, verbose=1)
  predictions.append(pred)
  
  if(i==10): predictions.append(Y[test])
  
  i+=1

print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



FOLD: 1
Epoch 1/20
191608/191608 [==============================] - 20s 104us/step - loss: 0.7413 - acc: 0.6180
Epoch 2/20
191608/191608 [==============================] - 9s 48us/step - loss: 0.6251 - acc: 0.7004
Epoch 3/20
191608/191608 [==============================] - 9s 48us/step - loss: 0.5608 - acc: 0.7281
Epoch 4/20
191608/191608 [==============================] - 9s 47us/step - loss: 0.5303 - acc: 0.7417
Epoch 5/20
191608/191608 [==============================] - 9s 48us/step - loss: 0.5004 - acc: 0.7524
Epoch 6/20
191608/191608 [==============================] - 9s 48us/step - loss: 0.4823 - acc: 0.7607
Epoch 7/20
191608/191608 [==============================] - 9s 48us/step - loss: 0.4695 - acc: 0.7707
Epoch 8/20
191608/191608 [==============================] - 9s 47us/step - loss: 0.4642 - acc: 0.7757
Epoch 9/20
191608/191608 [==============================] - 9s 47us/step - loss: 0.4527 - acc: 0.7780
Epoch 10/20
191608/191608 [==============================] - 9s 48us/st

In [0]:
#model.save('my_model1_cv.h5')

#model = load_model('my_model1_cv.h5')